In [2]:
from numpy import *
from scipy import misc
import PIL
from scipy.stats import norm
import matplotlib.pyplot as plt
import math
from sklearn.feature_extraction import image
import time
import pylab as plt1
from test import *
#from patchmatch_code import *
import cv2

w_patch=3
m_patch = 2*w_patch+1
knn=1

def offset_dict(offset, im, im_t):
    offset.astype(int)
    [M, N] = im.shape
    [M_t, N_t] = im_t.shape
    n = M*N
    n_t = M_t*N_t
    offsets = {}
    for i in range(0,n):
        idx = i + offset[i]
        idx = max(0, idx)
        idx = min(idx, n_t-1)
        j = int(idx)
        if j in offsets:
            offsets[j].append(i)
        else:
            offsets[j] = []
            offsets[j].append(i)
    return offsets


def color(idx, im, im_t, offsets, offsets_t):
    [M, N] = im.shape
    [M_t, N_t] = im_t.shape
    n = M*N
    n_t = M_t*N_t
    [i, j] = idx1d_to_idx2d(idx, M_t, N_t)
    co1=0.0
    sum1=0.0
    for r in range( max(0, i-w_patch), min(i+w_patch+1, M_t) ):
        for c in range( max(0, j-w_patch), min(j+w_patch+1, N_t) ):
            co1+=1
            idx1 = idx2d_to_idx1d(r, c, M_t, N_t) #pacth idx1 containing idx
            idx2 = idx1 + offsets_t[idx1] #patch in source which is nn of idx1 patch
            idx2 = max(0, idx2)
            idx2 = min(idx2, n-1)
            [i2, j2] = idx1d_to_idx2d(idx2, M, N)
            [i3, j3] = [i2+(r-i), j2+(c-j)]
            i3 = max(0, i3)
            i3 = min(i3, M-1)
            j3 = max(0, j3)
            j3 = min(j3, N-1)
            sum1+= im[i3, j3]
    sum1/= n_t
    co2=0.0
    sum2=0.0

    for r in range( max(0, i-w_patch), min(i+w_patch+1, M_t) ):
        for c in range( max(0, j-w_patch), min(j+w_patch+1, N_t) ):
            idx1 = idx2d_to_idx1d(r, c, M_t, N_t) #pacth idx1 containing idx
            if idx1 in offsets:
                for t in range(0, len(offsets[idx1]) ):
                    co2+=1
                    idx2 = offsets[int(idx1)][t] #patch in source which is nn of idx1 patch
                    idx2 = max(0, idx2)
                    idx2 = min(idx2, n-1)
                    [i2, j2] = idx1d_to_idx2d(idx2, M, N)
                    [i3, j3] = [i2+(r-i), j2+(c-j)]
                    i3 = max(0, i3)
                    i3 = min(i3, M-1)
                    j3 = max(0, j3)
                    j3 = min(j3, N-1)
                    sum2+= im[i3, j3]
    sum2/= n
    return (sum1+sum2)/(co1/n_t + co2/n)

def updation(im, im_t, offsets, offsets_t):
    [M, N] = im.shape
    [M_t, N_t] = im_t.shape
    n = M*N
    n_t = M_t * N_t
    for i in range(0,n_t):
        [i0, j0] = idx1d_to_idx2d(i, M_t, N_t)
        im_t[i0,j0] = color(i, im, im_t, offsets, offsets_t)
    return im_t

def iterations(im):
        c=0
        im_t = im
    #while im_t.shape[0]/float(im.shape[0]) >= 0.75 and im_t.shape[1]/float(im.shape[1]) >= 0.75:
        c+=1
        print(str(c)+"-iteration started")
        t0 = time.time()
        im_t = misc.imresize(im_t, 0.65)
        im_t = 255.*im_t/im_t.max()
        offset = patch_match(im_t, im, m_patch, knn)
        offsets_t = patch_match(im, im_t, m_patch, knn)
        offsets = offset_dict(offset, im, im_t)
        im_t = updation(im, im_t, offsets, offsets_t)
        for iter in range(3):
            print(iter)
            offset = patch_match(im_t, im, m_patch, knn)
            offsets_t = patch_match(im, im_t, m_patch, knn)
            offsets = offset_dict(offset, im, im_t)
            im_t = updation(im, im_t, offsets, offsets_t)
            dpi = 80.0
            plt.figure(figsize=(im_t.shape[0]/dpi, im_t.shape[1]/dpi), dpi=dpi)
            imgplot=plt.imshow(im_t, cmap='gray', vmin = 0, vmax = 255)
            #plt.show()
            plt.imshow('image',im_t)
            cv2.imwrite('figure'+str(c)+str(iter)+'.png',im_t)
            #cv2.destroyAllWindows()
        t1 = time.time()
        print(str(c)+"-iteration took " + str(t1-t0) + " s. \n")
        dpi = 80.0
        plt.figure(figsize=(im_t.shape[0]/dpi, im_t.shape[1]/dpi), dpi=dpi)
        imgplot=plt.imshow(im_t, cmap='gray', vmin = 0, vmax = 255)
        #plt.show()
        plt.imshow('image',im_t)
        cv2.imwrite('figure'+str(c)+'.png',im_t)
        # cv2.destroyAllWindows()
        #plt.imsave(im_t, cmap='gray', vmin = 0, vmax = 255)




In [ ]:
#if __name__ == "__main__":

t0 = time.time()
im_name = 'img/ours_10_aaa.jpg'
img = cv2.imread('img/ours_10_aaa.jpg',0)
im = cv2.imread(im_name, 0).astype("uint8")
im = 255.*im/im.max()
dpi = 80.0
#imgplot = plt.imshow(im)
#plt.show()

c=0
plt.figure(figsize=(im.shape[0]/dpi, im.shape[1]/dpi), dpi=dpi)
imgplot=plt.imshow(im, cmap='gray', vmin = 0, vmax = 255)
plt.imshow(im)
cv2.imwrite('figure'+str(c)+'.png',im)
#cv2.destroyAllWindows()
#misc.imsave("out.png", im,cmap='gray', vmin = 0, vmax = 255)
#plt.save(im, cmap='gray', vmin = 0, vmax = 255)
iterations(im)
t1 = time.time()
print("Total Execution took " + str(t1-t0) + " s. \n")
plt.show()

1-iteration started


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:100: DeprecationWarning:     `imresize` is deprecated!
    `imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
    Use ``skimage.transform.resize`` instead.


(354294, 7, 7)
(393216, 7, 7)
0
(354294, 7, 7)
(393216, 7, 7)


In [1]:
# -*- coding: utf-8 -*-

from numpy import *
from scipy.stats import norm
import math
from sklearn.feature_extraction import image
import time
import heapq

def dissimilarity_weights(m):
    w = int((m-1)/2)
    v = range(-w, w+1)
    [x, y] = meshgrid(v, v)
    g=norm.pdf(x)*norm.pdf(y)
    return g

def patches_dissimilarity(p1, p2, alpha, g):
    d2 = (p1-p2)**2
    weighted_d2 = g*d2
    dissim = -alpha*weighted_d2.sum()
    return dissim

def idx1d_to_idx2d(idx, M, N):
    j = int(floor(idx/M))
    i = idx - j*M
    return [i, j]

def idx2d_to_idx1d(i, j, M, N):
    idx = int(M*j + i)
    return idx


def try_improve_a_from_b(idx_a, idx_b, heap_a, heap_b,
                         patches_t, patches, M, N, alpha, g):
    k = len(heap_a)
    p0 = patches_t[idx_a, :, :]# patch around pixel idx_a
    for nn in range(0,k):
        # compute the 2D offset corresponding idx_b's nn-est nearest neighbour
        offs_b = heap_b[nn][1]
        idx_d = int(idx_a + offs_b)
        idx_d = max(idx_d, 0)
        idx_d = min(idx_d, patches.shape[0]-1)
        offs_b = idx_d - idx_a
        p2 = patches[idx_d, :, :]# patch around the new pixel to compare to idx_a
        w_b = patches_dissimilarity(p0, p2, alpha, g)# new weight
        if w_b > heap_a[0][0] and not ((w_b, offs_b) in heap_a):
            heapq.heapreplace(heap_a, (w_b, offs_b))
    return heap_a



def build_new_offsets(idx, heap0, L, l, q, M, N):
    new_offsets = zeros(len(heap0))
    for k in range(0, len(heap0)):
        idx2 = int(idx) + int(heap0[k][1])
        [i2, j2] = idx1d_to_idx2d(idx2, M, N)
        [u, v] = (L*l**q)*(2*random.rand(2,) -1)
        [i3, j3] = array([i2, j2]) + array([u, v])
        i3 = int(i3)
        j3 = int(j3)
        i3 = max(0, i3)
        i3 = min(M-1, i3)
        j3 = max(0, j3)
        j3 = min(N-1, j3)
        idx3 = idx2d_to_idx1d(i3, j3, M, N)
        new_offsets[k] = idx3 - idx
    return new_offsets


def search_around(idx, heap0, patches_t, patches, M, N, alpha, g):
    L = 30
    l= 0.5
    q_range = arange(0, 3)
    cand_offsets = empty((q_range.shape[0], len(heap0)))
    c = 0
    for q in q_range:
        cand_offsets[c,] = build_new_offsets(idx, heap0, L, l, q, M, N)
        c = c+1
    for c in range(0,3):
        heap1 = []
        for k in range(0, len(heap0)):
            heap1.append((0, cand_offsets[c, k]))
        new_heap0 =  try_improve_a_from_b(idx, idx, heap0,
                                          heap1, patches_t, patches,
                                          M, N, alpha, g)
    return new_heap0


def initialize_offsets(M_t, N_t, M, N, k):
    n_t = M_t*N_t
    n = M*N
    offsets = floor((n-1)*random.rand(n_t,k)) - tile(arange(0, n_t), (k, 1)).T
    offsets.astype(int)
    return offsets

def initialize_weights(patches_t, patches, offsets, alpha, g):
    [n_t, k] = offsets.shape
    weights = ones([n_t, k])
    for i in range(0, n_t):
        p0 = patches_t[i, :, :]
        for j in range(0, k):
            i2 = int(i + offsets[i, j])
            p2 = patches[i2, :, :]
            weights[i, j] = patches_dissimilarity(p0, p2, alpha, g)
    return weights

def initialize_heap(offsets, weights):
    all_heaps = []
    [n_t, k] = offsets.shape
    for i in range(0,n_t):
        h = []
        for j in range(0, k):
            heapq.heappush(h, (weights[i, j], offsets[i, j]))
        all_heaps.append(h)
    return all_heaps


def propagate(iter_n, current_heaps, M_t, N_t, M, N, patches_t, patches, alpha, g):
    if iter_n % 2:
        Jrange = range(1, N_t)
        Irange = range(1, M_t)
        pix_shift = -1
    else:
        Jrange = arange(N_t-2, -1, -1)
        Irange = arange(M_t-2, -1, -1)
        pix_shift = 1

    for j0 in Jrange:
        for i0 in Irange:
            idx_0 = idx2d_to_idx1d(i0, j0, M_t, N_t)
            idx_1 = idx2d_to_idx1d(i0+pix_shift, j0, M_t, N_t)
            idx_2 = idx2d_to_idx1d(i0, j0+pix_shift, M_t, N_t)
            heap0 = current_heaps[idx_0]
            heap1 = current_heaps[idx_1]
            heap2 = current_heaps[idx_2]
            heap0 = try_improve_a_from_b(idx_0, idx_1, heap0,
                                          heap1, patches_t, patches,
                                          M, N, alpha, g)
            current_heaps[idx_0] = try_improve_a_from_b(idx_0, idx_2, heap0,
                                                         heap2, patches_t, patches,
                                                         M, N, alpha, g)
    return current_heaps

def random_search(current_heaps, M_t, N_t, M, N, patches_t, patches, alpha, g):
    n_t = M_t*N_t
    new_heaps = []
    for idx in range(0, n_t):
        heap_0 = current_heaps[idx]
        new_heap0 = search_around(idx, heap_0, patches_t, patches, M, N, alpha, g)
        new_heaps.append(new_heap0)
    return new_heaps

def heaps_to_weights_and_offsets(all_heaps):
    n = len(all_heaps)
    k = len(all_heaps[0])
    offsets = zeros((n, k), dtype=int)
    weights = zeros((n, k))
    for i in range(0,n):
        for j in range(0,k):
            weights[i, j] = all_heaps[i][j][0]
            offsets[i, j] = int(all_heaps[i][j][1])
    offsets.astype(int)
    return offsets, weights

def patch_match(im, im_t, m, knn):
    w = int((m-1)/2)
    alpha = 0.01
    g = dissimilarity_weights(m)
    im_bis = pad(im, [w, w], 'symmetric')
    im_bis_t = pad(im_t, [w, w], 'symmetric')
    [M, N] = im.shape
    n = M*N
    [M_t, N_t] = im_t.shape
    n_t = M_t*N_t
    #get list of patches for source and target images
    patches = image.extract_patches_2d(im_bis.T, (m, m))
    for i in range(0,n):
        patches[i]=patches[i].T
    patches_t = image.extract_patches_2d(im_bis_t.T, (m, m))
    for i in range(0,n_t):
        patches_t[i]=patches_t[i].T
    print(patches.shape)
    offsets_init = initialize_offsets(M_t, N_t, M, N, knn)
    weights_init = initialize_weights(patches_t, patches, offsets_init, alpha, g)
    all_heaps = initialize_heap(offsets_init, weights_init)
    new_heaps = propagate(1, all_heaps, M_t, N_t, M, N, patches_t, patches, alpha, g)
    new_heaps = random_search(new_heaps, M_t, N_t, M, N, patches_t, patches, alpha, g)
    new_heaps = propagate(2, new_heaps, M_t, N_t, M, N, patches_t, patches, alpha, g)
    offsets, weights = heaps_to_weights_and_offsets(all_heaps)
    return offsets

In [10]:
!pip3 install --upgrade scipy==1.1.0

     |████████████████████████████████| 31.2 MB 2.5 MB/s eta 0:00:01     |███████████████████████         | 22.4 MB 1.9 MB/s eta 0:00:05     |██████████████████████████▉     | 26.2 MB 2.2 MB/s eta 0:00:03
  Attempting uninstall: scipy
    Found existing installation: scipy 1.5.4
    Uninstalling scipy-1.5.4:
      Successfully uninstalled scipy-1.5.4
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [18]:
! pip3 install imageio

     |████████████████████████████████| 3.3 MB 2.6 MB/s eta 0:00:01     |██████████████▋                 | 1.5 MB 2.5 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [19]:
import imageio

In [3]:
plt.show()